In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


In [14]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')

In [104]:
stop_words = stop_words+['ресторан', 'интерьер', 'это', 'просто', 'очень', 'заведение', 'наш', 'свой', 'пока', 'который']

In [4]:
import re
import pymorphy2 as pm2 
pmm = pm2.MorphAnalyzer() 

In [24]:
def cleanText(text):
    text = text.lower()
    text = re.sub('-', ' ', text)
    text = re.sub(r'[^\w\s]','',text) 
    text = re.sub(r'\d', '', text) 
    text = re.sub(r'[A-Za-z]', '', text)
    text = [pmm.normal_forms(x)[0] for x in text.split() if x not in stop_words] 
    for i in text:
        if i in stop_words:
            text.remove(i)
    return text

In [48]:
for file in ['37516.txt', '26330.txt','30808.txt', '33838.txt']:
    with open (file, encoding='utf-8') as f:
        content = f.read()
    cleaned = cleanText(content)
    data.append(cleaned)

In [25]:
data = []

In [175]:
with open ('SentiRuEval_rest_train.xml', encoding='utf-8') as f:
    content = f.read()
    needed = re.findall('<text>.*?</text>', content)
    n = [re.sub('</?text>', '', k) for k in needed[:5000]]
    clean = [cleanText(content) for content in n]

In [164]:
len(needed)

19034

In [176]:
data = clean

In [177]:
id2word = corpora.Dictionary(data)

In [178]:
corpus = [id2word.doc2bow(text) for text in data]

In [179]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [180]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.013*"салат" + 0.011*"соус" + 0.010*"суп" + 0.009*"блюдо" + 0.008*"вкус" + '
  '0.007*"мясо" + 0.005*"меню" + 0.005*"тарелка" + 0.004*"овощ" + 0.004*"р"'),
 (1,
  '0.006*"краб" + 0.005*"вафля" + 0.004*"мыть" + 0.003*"кальмар" + '
  '0.003*"висеть" + 0.003*"цвет" + 0.002*"прибыть" + 0.002*"рэ" + '
  '0.002*"стекло" + 0.002*"порог"'),
 (2,
  '0.015*"понравиться" + 0.015*"кухня" + 0.014*"место" + 0.014*"хороший" + '
  '0.013*"вкусный" + 0.011*"весь" + 0.009*"приятный" + 0.009*"обслуживание" + '
  '0.008*"вкусно" + 0.008*"большой"'),
 (3,
  '0.008*"сказать" + 0.008*"официант" + 0.008*"меню" + 0.007*"принести" + '
  '0.007*"стол" + 0.006*"человек" + 0.006*"столик" + 0.006*"заказать" + '
  '0.005*"официантка" + 0.005*"девушка"')]
